In [18]:
# -*- coding: utf-8 -*-

###### reference:
 1. 觀測資料查詢
  + https://e-service.cwb.gov.tw/HistoryDataQuery/
 - 地震中心
  + https://scweb.cwb.gov.tw/Page.aspx?ItemId=20&loc=tw&adv=1
 - 20180425_觀測站URL翻譯
  + http://127.0.0.1:7777/notebooks/%E6%95%B8%E6%93%9A%E7%A7%91%E5%AD%B8%E6%9C%9F%E6%9C%AB%E5%A0%B1%E5%91%8A/%E9%81%8E%E5%8E%BB%E5%8F%83%E8%80%83/20180425_%E8%A7%80%E6%B8%AC%E7%AB%99URL%E7%BF%BB%E8%AD%AF.ipynb
 - python dict
  + http://www.runoob.com/python/python-dictionary.html
 - What is the correct way to select an option using Selenium's Python WebDriver
  + https://sqa.stackexchange.com/questions/1355/what-is-the-correct-way-to-select-an-option-using-seleniums-python-webdriver
 - beautifulsoup find_all
  + https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/
 - json dump save
  + http://kuma-uni.blogspot.com/2012/06/jsonpythonjson.html
  + https://docs.python.org/2/library/json.html
  + https://stackoverflow.com/questions/7100125/storing-python-dictionaries

In [2]:
import urllib 
from bs4 import BeautifulSoup
from selenium import webdriver
import json
import time

###### 調用函數

In [5]:
#select 點擊
def multiselect_set_selections(driver, element_id, labels):
    el = driver.find_element_by_id(element_id)
    for option in el.find_elements_by_tag_name('option'):
        if option.text in labels:
            option.click()
    return

#### 小目標
 - 蒐集各縣市主要觀測站資料
  + 測站縣市、測站編號、測站名字

### 主要流程

指定瀏覽器，並開起指定網頁

資料儲存變數

流程

In [ ]:
def Crawler_station(driver, base_url):
    stationDict = dict()
    #取得
    driver.get(base_url)#+"index.jsp")
    time.sleep(5) #等五秒，網站有點 LAG
    # 全名列表，換"測站所在縣市用"， labels
    cityHoleNameList = list()
    # 用於設定主要測站
    cityFirst = True
    #撈文章列表
    page_html    = driver.page_source
    page_soup    = BeautifulSoup(page_html,'html.parser')
    page_findTAG = page_soup.find_all('select')
    # 縣市列表，建表
    for temp in page_findTAG[0].find_all('option'):
        cityHoleNameList.append(temp.text)
        stationDict[str(temp.text[:3])] = dict()
    for city in cityHoleNameList:
        multiselect_set_selections(driver, "stationCounty", city)
        # 撈新頁面
        page_html    = driver.page_source
        page_soup    = BeautifulSoup(page_html,'html.parser')
        page_findTAG = page_soup.find_all('select')
        # 取得觀測站名稱及編號
        for stationStr in page_findTAG[1].find_all('option'):
            #print(stationStr)
            valueStr = str(stationStr).split('"')[1]
            nameStr = stationStr.text.split(" ")[0]
            code = urllib.parse.quote(urllib.parse.quote(nameStr))
            stationDict[city[:3]][valueStr] = code
            if cityFirst:
                stationDict[city[:3]]["stationNo"] =valueStr
                stationDict[city[:3]]["stationName"] = code
                cityFirst = False
        cityFirst = True 
    return stationDict

# MAIN

In [ ]:
#變數
base_url = "https://e-service.cwb.gov.tw/HistoryDataQuery/"
stationDictJson = "cityStationData.json"

driver = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe')

stationDict = Crawler_station(driver, base_url)

driver.quit() #driver.close()


In [15]:
with open(stationDictJson, 'w') as outfile:
    json.dump(stationDict, outfile, sort_keys=True)
    #outfile.write(json.dumps(stationDict, sort_keys = True ))

In [16]:
with open(stationDictJson, 'r') as inputfile:
    decodejson =  json.load(inputfile)

In [17]:
decodejson

{'南投縣': {'467550': '%25E7%258E%2589%25E5%25B1%25B1',
  '467650': '%25E6%2597%25A5%25E6%259C%2588%25E6%25BD%25AD',
  'C0H890': '%25E5%259F%2594%25E9%2587%258C',
  'C0H950': '%25E4%25B8%25AD%25E5%25AF%25AE',
  'C0H960': '%25E8%258D%2589%25E5%25B1%25AF',
  'C0H990': '%25E6%2598%2586%25E9%2599%25BD',
  'C0H9A0': '%25E7%25A5%259E%25E6%259C%25A8%25E6%259D%2591',
  'C0H9C0': '%25E5%2590%2588%25E6%25AD%25A1%25E5%25B1%25B1',
  'C0I010': '%25E5%25BB%25AC%25E5%25B1%25B1',
  'C0I080': '%25E4%25BF%25A1%25E7%25BE%25A9',
  'C0I090': '%25E9%25B3%25B3%25E5%2587%25B0',
  'C0I110': '%25E7%25AB%25B9%25E5%25B1%25B1',
  'C0I360': '%25E6%25B0%25B4%25E9%2587%258C',
  'C0I370': '%25E9%25AD%259A%25E6%25B1%25A0',
  'C0I380': '%25E9%259B%2586%25E9%259B%2586',
  'C0I390': '%25E4%25BB%2581%25E6%2584%259B',
  'C0I410': '%25E5%2590%258D%25E9%2596%2593',
  'C0I420': '%25E5%259C%258B%25E5%25A7%2593',
  'C0I460': '%25E5%258D%2597%25E6%258A%2595',
  'C0I480': '%25E6%25A2%2585%25E5%25B3%25B0',
  'C0I490': '%25E8%2590%25AC